In [29]:
import os
import json
from pathlib import Path
import pandas as pd
import pymupdf
from tqdm.notebook import tqdm
import random
import jinja2
import string
import textwrap
from openai import OpenAI
from dotenv import load_dotenv

In [30]:
with open("cpv_divisions.json", "r", encoding="utf-8") as file:
    cpv_mapping = json.load(file)

In [31]:
current_dir = Path(os.getcwd())
pdf_paths = list(current_dir.rglob("*.pdf"))

data = {
    "file": [],
    "text": [],
    "page": [],
    "division_number": [],
    "division_title": [],
    "language": [],    
}

for p in tqdm(pdf_paths):
    try:
        cft_root = p.parts.index("docs") - 1
        file = "/".join(p.parts[cft_root:])
        
        doc = pymupdf.open(p)

        for idx, page in enumerate(doc):
            data["file"].append(file)
    
            data["text"].append(page.get_text())
        
            data["page"].append(idx+1)
            
            division_number = int(p.parts[p.parts.index("data") + 1])
            data["division_number"].append(division_number)
        
            division_title = cpv_mapping.get(str(division_number), None)
            data["division_title"].append(division_title)
        
            lang = p.parts[p.parts.index("data") + 2]
            data["language"].append(lang)

    except:
        print("Could not open file", file)

df = pd.DataFrame(data)
df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,file,text,page,division_number,division_title,language
0,169436/docs/180404_it_oetwil.pdf,\nSubmissionsbeschrieb \nIT-Beschaffung Schul...,1,30,"Office and computing machinery, equipment and ...",DE
1,169436/docs/180404_it_oetwil.pdf,\n \n \n \nSeite 2 von 5 \nEignungskriterien:...,2,30,"Office and computing machinery, equipment and ...",DE
2,169436/docs/180404_it_oetwil.pdf,\n \n \n \nSeite 3 von 5 \nBeschaffungsliste ...,3,30,"Office and computing machinery, equipment and ...",DE
3,169436/docs/180404_it_oetwil.pdf,\n \n \n \nSeite 4 von 5 \nPosition 7 \n174 S...,4,30,"Office and computing machinery, equipment and ...",DE
4,169436/docs/180404_it_oetwil.pdf,\n \n \n \nSeite 5 von 5 \nPosition 14 \n8 St...,5,30,"Office and computing machinery, equipment and ...",DE


In [32]:
print(df.sample().text.item())

 
 
 
 
Seite 2 von 5 
Eignungskriterien: 
- Anbieter ist in der Lage, das geforderte Volumen 
termingerecht anzubieten. 
- Anbieter hat die organisatorische, technische, 
personelle und finanzielle Leistungsfähigkeit. 
- Anbieter reicht ein vollständiges Angebot gemäss 
den Anforderungen ein. 
- Anbieter kann die erforderlichen Nachweise 
vorlegen. 
- Termingerechte Eingabe des Angebots. 
Nachweise: 
- Referenzen 
- Betreibungsauszug 
- Stellenprozenten total im Betrieb 
- Anzahl Lehrlinge im Betrieb 
Zuschlagskriterien: 
Preis 70% 
Qualität des Angebotes 20% 
Qualität des Anbieters 5% 
Lehrlingsausbildung 5% 
Varianten und 
Teilangebote: 
Varianten und Teilangebote sind nicht zulässig. 
Aufbau der Offerte: 
Um eine Vergleichbarkeit der Offerten zu 
gewährleisten, muss sich die Offerte nach den in 
diesen Unterlagen aufgeführte Positionen richten. 
 
Die Kosten sind für jede im Pflichtenheft aufgeführte 
Position separat in Schweizerfranken und ohne 
Mehrwertsteuer auszuweisen. Im Tot

In [33]:
def generate_prompt(text, language):
    option_amt = random.randint(3, 7)
    option_labels = list(string.ascii_uppercase[:option_amt])
    
    template_str = textwrap.dedent(
        """\
        Based on the following text, generate one multiple-choice question with {{ option_amt }} options. Indicate the correct option explicitly.
        Language of the text and your output: {{ language }}
        The multiple-choice question and answer needs to fulfill the following criteria:
        Self-contained: The correct answer can be directly derived from the text without prior knowledge.
        Distractors: The incorrect options are plausible but contradicted by specific details in the text.
        Clarity: Each option references a specific, verifiable fact from the provided information.
        
        Text:
        {{ text }}

        Output Format (in JSON):
        {
            "question": "<Your question here>",
            "options": {
                {% for label in option_labels %}\
                "{{ label }}": "Option {{ loop.index }}"{% if not loop.last %},{% endif %}
                {% endfor %}\
            },
            "correct_answer": "{{ correct_label }}"
        }
        """
    )
    template = jinja2.Template(template_str)
    
    prompt = template.render(
        option_amt=option_amt,
        text=text,
        option_labels=option_labels,
        language= language
    )
    
    return prompt

In [34]:
# takes a list of prompts, sends them to the LLM and returns the chat completions.
def ask_LLM_and_extract_MQC(prompts:list):
    mcq_list = []
    for prompt, language in prompts:
        load_dotenv()
        deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")
        deepinfra_base_url = os.getenv("DEEPINFRA_BASE_URL")
        openai = OpenAI(api_key=deepinfra_api_key,
                        base_url=deepinfra_base_url)

        response = openai.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-405B-Instruct",
                        messages=[
                            {"role": "system", "content": "You are a knowledgeable assistant that only responds in the given output format."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=1500,
                        temperature=0.7,
                    ).choices[0].message.content.strip()
        mcq_question = extract_MCQ(response)
        if not mcq_question: # give the LLM an oppurtuinity to correct itself
            response = openai.chat.completions.create(
                        model="meta-llama/Meta-Llama-3.1-405B-Instruct",
                        messages=[
                            {"role": "system", "content": "You are a knowledgeable assistant that only responds in the given output format."},
                            {"role": "user", "content": prompt},
                            {"role": "assistant", "content": response},
                            {"role": "user", "content": "I was unable to convert your response into a json file. There is an issue in the format of your response. Please provide the multiple-choice question again, this time in correct JSON Format."}
                        ],
                        max_tokens=1500,
                        temperature=0.7,
                    ).choices[0].message.content.strip()
            mcq_question = extract_MCQ(response)
        if mcq_question:
            mcq_question["language"] = language
            mcq_list.append(mcq_question)
        else:
            print("Unable to provide a MCQ.")

    return mcq_list

def extract_MCQ(response):
    # Extracting MCQ from the answer.
    start = response.find("{")
    end = response.rfind("}")
    if start != -1 and end != -1:  # Ensure both are found
        json_content = response[start:end+1]  
        try:
            mcq_data = json.loads(json_content)
            return mcq_data
        except json.JSONDecodeError:
            return False  
    return False 

def save_to_dict(mcq_list, file_name="LLM_generated_multiple_choice_question.json"):
    path = "output/" + file_name
    with open(path, "w", encoding="utf-8") as f:
        json.dump(mcq_list, f, ensure_ascii=False, indent=4)
    print(f"Multiple-choice question saved to {path}")

In [35]:
prompts = []
samples = df.to_dict(orient='records')
for sample in samples:
    prompt = generate_prompt(sample['text'], sample['language'])
    prompts.append((prompt, sample['language']))

mcq_list = ask_LLM_and_extract_MQC(prompts)
save_to_dict(mcq_list)

Multiple-choice question saved to output/LLM_generated_multiple_choice_question.json


In [42]:
def shuffle_mcqs(mcq):
    shuffled_mcqs = []
    
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']
    
    for _ in range(4):
        options_values = list(options.values()) 
        random.shuffle(options_values)
        shuffled_options = {label: options_values[i] for i, label in enumerate(options.keys())}
        new_correct_answer = next(label for label, option in shuffled_options.items() if option == options[correct_answer])
    
        shuffled_mcqs.append({
            'question': question,
            'options': shuffled_options,
            'correct_answer': new_correct_answer,
            'version': "reordered",
            'language': language
        })
    
    return shuffled_mcqs

def none_of_the_above_mcqs(mcq):    
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']

    if language == 'DE':
        nota_str = "Keine der oben genannten Optionen"
    else:
        nota_str = "None of the above"
    options_values = list(options.values()) 
    options_values[len(options_values)-1] = nota_str
    options = {label: options_values[i] for i, label in enumerate(options.keys())}
    
    nota_mcq = {
        'question': question,
        'options': options,
        'correct_answer': correct_answer,
        'version': "nota",
        'language': language
    }
    
    return nota_mcq

def true_false(mcq):
    question = mcq['question']
    options = mcq['options']
    correct_answer = mcq['correct_answer']
    language = mcq['language']

    options_values = list(options.values())
    answer = random.choice(options_values)
    label = False
    if options[correct_answer] == answer:
        label = True

    if language == 'DE':
        true_false_prompt = "Ist das folgende Frage-Antwort Paar Richtig oder Falsch?"
    else: 
        true_false_prompt = "Is the following question-answer pair True or False?"

    true_false_question = {
        "true_false_prompt": true_false_prompt,
        "question": question,
        "answer": answer,
        "label": label,
        "version": "true_false",
        "language": language
    }
    return true_false_question

def enhance_mcqs(mcq_list):
    mcqs_plus = []
    for mcq in mcq_list:
        mcq_plus = shuffle_mcqs(mcq)
        mcq_plus[3] = none_of_the_above_mcqs(mcq_plus[3])
        mcq_plus.append(true_false(mcq))
        mcqs_plus.append(mcq_plus)
    return mcqs_plus

file_name = "output/LLM_generated_multiple_choice_question.json"
with open(file_name, "r") as f:
    mcq_list = json.load(f)

enhanced_mcqs = enhance_mcqs(mcq_list)
save_to_dict(enhanced_mcqs,file_name="enhanced.json")

Multiple-choice question saved to output/enhanced.json
